# Teleportation #

In [ ]:
# imports
import os;
import sys;

# NOTE: need this to force jupyter to reload imports:
for key in list(sys.modules.keys()):
    if key.startswith('src.'):
        del sys.modules[key];

os.chdir(os.path.dirname(_dh[0]));
sys.path.insert(0, os.getcwd());

from src.thirdparty.maths import *;
from src.thirdparty.quantum import *;
from src.demo.teleportation import *;

np.random.seed(39102901); # for repeatability

In [ ]:
action_display_circuit();

In [ ]:
action_prepare_circuit_and_job(
    option = BACKEND_SIMULATOR.AER,
    num_samples = 100,
    num_shots = 100,
);
action_display_statistics();

In [ ]:
action_prepare_circuit_and_job(
    option = BACKEND.LEAST_BUSY,
    num_samples = 10, # <- do not set this too high!
    num_shots = 100,
);

In [ ]:
action_display_statistics(queue=True);